## Task 4

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras import layers, Input
from random import *
import pandas as pd

In [2]:
tf.test.is_gpu_available(cuda_only=True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [14]:
IMG_SIZE = 128
train = pd.read_fwf('train_triplets.txt', header=None, dtype=str)
maxRows = train.size/3 - 1

In [23]:
def image_generator(bs):
    
    j = 0
    while True:
        X = []
        Y = []
        
        while len(X) < bs:
            if (j >= maxRows):
                j = 0
            else :
                path1 = "/home/nicolas/eth/task4/food/" + j + '.jpg'
                img = cv2.imread(path1)
                imgReshaped = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                imgScaled = imgReshaped.astype('float32')

                X.append(imgScaled)
                Y.append([1,0])
                
            j = j + 1

        yield (np.array(X),np.array(Y))

In [23]:
# Download the Xception model
sharedModelXception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE,IMG_SIZE,3),
)

sharedModelXception.trainable = False

In [24]:
# Construct the CNN
# https://link.springer.com/article/10.1007/s00521-019-04242-5?shared-article-renderer
x = keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))

k = tf.keras.applications.xception.preprocess_input(x)

out_x = sharedModelXception(k, training=False)

poolx = layers.GlobalAveragePooling2D()(out_x)

model = keras.Model(
    inputs=x,
    #outputs=[normalizedX,normalizedY,normalizedZ],
    outputs=poolx,
)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 128, 128, 3)]     0         
_________________________________________________________________
xception (Model)             (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 20,861,480
Trainable params: 0
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
features = pd.DataFrame()
j = 0

for j in range(0,10000):
    path = "/home/nicolas/eth/task4/food/" + str(j).zfill(5) + '.jpg'
    img = cv2.imread(path)
    imgReshaped = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    imgScaled = imgReshaped.astype('float32')

    l = np.array(imgScaled).reshape(1,IMG_SIZE,IMG_SIZE,3)
    
    encoding = model.predict(l)
    
    print(type(encoding[0]))

    rowdf = pd.DataFrame([encoding[0]])
    features = pd.concat([features,rowdf],ignore_index=True)


In [80]:
features

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,1.101184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353460,0.226726,...,0.023228,0.615383,2.083772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016415
1,0.906689,1.562713,0.000000,0.000000,0.429174,0.005077,0.030905,0.265437,0.060025,0.022515,...,0.044595,1.502733,0.198539,0.015324,0.026071,0.000000,0.005770,0.000000,0.412423,0.000000
2,0.004670,0.011644,0.252479,0.000000,0.000000,0.004107,0.056194,0.000000,0.000000,0.000000,...,0.316784,1.071527,0.000000,0.000000,0.359007,1.666463,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.345675,0.000000,0.000000,0.004943,0.000000,0.545909,0.208788,0.267135,0.004936,...,0.508176,0.971396,1.011045,0.456809,0.000000,0.291129,0.000000,0.000000,0.000098,0.338569
4,0.000000,0.650283,0.510727,0.336428,0.424996,0.923469,0.005860,0.715334,1.663253,0.000000,...,0.000000,0.350181,1.718871,0.742059,1.153492,0.000000,0.082586,0.188616,0.239336,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.049212,0.452515,0.000000,0.002544,0.244619,0.886274,0.010726,0.000943,2.577320,0.000000,...,0.644293,0.000000,1.949005,0.002226,0.000000,0.156330,0.691592,0.071347,0.000000,0.040373
9996,0.260263,1.127068,0.150439,0.000000,0.000000,0.375883,0.001233,0.000000,0.189791,0.351563,...,1.134800,0.000000,1.510809,0.023706,0.000000,0.004772,0.000000,0.000000,0.000000,0.000000
9997,0.000000,0.008660,0.000000,0.000000,0.000000,0.150254,0.225151,0.000000,1.539958,0.016108,...,0.000000,0.477397,0.006090,0.000000,0.004094,0.031761,0.333861,0.000000,0.000000,0.000000
9998,0.374892,0.713724,0.002749,0.000000,0.000000,0.016238,1.660373,0.000000,0.580271,0.000000,...,0.042421,0.588791,0.000000,0.000000,0.000000,1.264809,0.000000,0.854869,0.000000,0.000000


In [81]:
for x in range(0,len(features.columns)):
    mean = features.iloc[:,x].mean()
    std = features.iloc[:,x].std()
    features.iloc[:,x] = (features.iloc[:,x]-mean)/std

In [82]:
features

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.642852,1.073921,-0.463742,-0.331542,-0.579972,-0.525070,-0.584284,-0.459762,-0.534061,0.136363,...,-0.648320,0.151033,1.942014,-0.451011,-0.476435,-0.511451,-0.592828,-0.479123,-0.598841,-0.453654
1,1.396597,1.873639,-0.463742,-0.331542,0.504022,-0.511158,-0.508117,0.227576,-0.943197,-0.452427,...,-0.606323,1.620106,-0.638160,-0.400750,-0.408806,-0.511451,-0.580754,-0.479123,0.118195,-0.496394
2,-0.632348,-0.813987,0.323736,-0.331542,-0.579972,-0.513816,-0.445792,-0.459762,-1.026890,-0.517344,...,-0.071331,0.906212,-0.909886,-0.451011,0.454851,3.483603,-0.592828,-0.479123,-0.598841,-0.496394
3,-0.642852,-0.235192,-0.463742,-0.331542,-0.567487,-0.525070,0.761128,0.080886,-0.654424,-0.503112,...,0.304853,0.740439,0.473854,1.047304,-0.476435,0.186480,-0.592828,-0.479123,-0.598671,0.385134
4,-0.642852,0.292620,1.129207,1.160978,0.493470,2.005527,-0.569841,1.392563,1.292182,-0.517344,...,-0.693975,-0.288029,1.442602,1.982911,2.515794,-0.511451,-0.419995,0.062904,-0.182733,-0.496394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.717179,-0.050064,-0.463742,-0.320256,0.037879,1.903602,-0.557848,-0.457320,2.566665,-0.517344,...,0.572392,-0.867779,1.757568,-0.443708,-0.476435,-0.136677,0.854526,-0.274092,-0.598841,-0.391275
9996,-0.057432,1.118772,0.005473,-0.331542,-0.579972,0.504970,-0.581245,-0.459762,-0.762265,0.496301,...,1.536492,-0.867779,1.157844,-0.373256,-0.476435,-0.500012,-0.592828,-0.479123,-0.598841,-0.496394
9997,-0.642852,-0.819158,-0.463742,-0.331542,-0.579972,-0.113325,-0.029391,-0.459762,1.120273,-0.470899,...,-0.693975,-0.077413,-0.901551,-0.451011,-0.465815,-0.435309,0.105871,-0.479123,-0.598841,-0.496394
9998,0.200408,0.402547,-0.455167,-0.331542,-0.579972,-0.480572,3.507764,-0.459762,-0.217819,-0.517344,...,-0.610597,0.107007,-0.909886,-0.451011,-0.476435,2.520706,-0.592828,1.977516,-0.598841,-0.496394


In [83]:
features.to_csv(r'/home/nicolas/eth/task4/features.csv',index=False,header=True)

## After the Encoding

In [4]:
features = pd.read_csv('features.csv')

In [5]:
features

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.642852,1.073921,-0.463742,-0.331542,-0.579972,-0.525070,-0.584284,-0.459762,-0.534061,0.136363,...,-0.648320,0.151033,1.942014,-0.451011,-0.476435,-0.511451,-0.592828,-0.479123,-0.598841,-0.453654
1,1.396597,1.873639,-0.463742,-0.331542,0.504022,-0.511158,-0.508117,0.227576,-0.943197,-0.452427,...,-0.606323,1.620106,-0.638160,-0.400750,-0.408806,-0.511451,-0.580754,-0.479123,0.118195,-0.496394
2,-0.632348,-0.813987,0.323736,-0.331542,-0.579972,-0.513816,-0.445792,-0.459762,-1.026890,-0.517344,...,-0.071331,0.906212,-0.909886,-0.451011,0.454851,3.483603,-0.592828,-0.479123,-0.598841,-0.496394
3,-0.642852,-0.235192,-0.463742,-0.331542,-0.567487,-0.525070,0.761128,0.080886,-0.654424,-0.503112,...,0.304853,0.740439,0.473854,1.047304,-0.476435,0.186480,-0.592828,-0.479123,-0.598671,0.385134
4,-0.642852,0.292620,1.129207,1.160978,0.493470,2.005527,-0.569841,1.392563,1.292182,-0.517344,...,-0.693975,-0.288029,1.442602,1.982911,2.515794,-0.511451,-0.419995,0.062904,-0.182733,-0.496394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.717179,-0.050064,-0.463742,-0.320256,0.037879,1.903602,-0.557848,-0.457320,2.566665,-0.517344,...,0.572392,-0.867779,1.757568,-0.443708,-0.476435,-0.136677,0.854526,-0.274092,-0.598841,-0.391275
9996,-0.057432,1.118772,0.005473,-0.331542,-0.579972,0.504970,-0.581245,-0.459762,-0.762265,0.496301,...,1.536492,-0.867779,1.157844,-0.373256,-0.476435,-0.500012,-0.592828,-0.479123,-0.598841,-0.496394
9997,-0.642852,-0.819158,-0.463742,-0.331542,-0.579972,-0.113325,-0.029391,-0.459762,1.120273,-0.470899,...,-0.693975,-0.077413,-0.901551,-0.451011,-0.465815,-0.435309,0.105871,-0.479123,-0.598841,-0.496394
9998,0.200408,0.402547,-0.455167,-0.331542,-0.579972,-0.480572,3.507764,-0.459762,-0.217819,-0.517344,...,-0.610597,0.107007,-0.909886,-0.451011,-0.476435,2.520706,-0.592828,1.977516,-0.598841,-0.496394


In [129]:
from random import *
IMG_SIZE = 128
BS = 32

train = pd.read_fwf('train_triplets.txt', header=None, dtype=str)

In [170]:
with open('train_triplets.txt', 'r') as f:
    lines = f.readlines()

triplets = []
for line in lines:
    a1 = int(line[0:5])
    a2 = int(line[6:11])
    a3 = int(line[12:17])
    arr = [a1,a2,a3]
    triplets.append(arr)

def triplet_in_set(triplet, myset):
    t1 = triplet[0] in myset
    t2 = triplet[1] in myset
    t3 = triplet[2] in myset
    return t1 and t2 and t3

from random import seed
from random import random
seed(123)
traindata = []
trainset = set([])
validata = []
valiset = set([])
discardcount = 0
for triplet in triplets:
    if not triplet_in_set(triplet, trainset) and not triplet_in_set(triplet, valiset):
        if random() <= 0.65:
            traindata.append(triplet)
            trainset = trainset.union(set(triplet))
        else:
            validata.append(triplet)
            valiset = valiset.union(set(triplet))
    elif not triplet_in_set(triplet, valiset):
        traindata.append(triplet)
        trainset = trainset.union(set(triplet))
    elif not triplet_in_set(triplet, trainset):
        validata.append(triplet)
        valiset = valiset.union(set(triplet))
    else:
        discardcount += 1
print(discardcount)
print(len(traindata))
print(len(validata))

12155
46243
1117


In [171]:
numSamplesTrain = len(traindata)
maxSamplesTrain = numSamples*2
print(numSamplesTrain)

46243


In [172]:
numSamplesEval = len(validata)
maxSamplesEval = numSamples*2
print(numSamplesEval)

1117


In [173]:
def image_generator(bs):
    
    j = 0
    while True:
        X = []
        Y = []
        
        while len(X) < bs:
            if (j >= numSamplesTrain):
                j = 0
            else :
                a,b,c = traindata[j]

                enc1 = features.iloc[a]
                enc2 = features.iloc[b]
                enc3 = features.iloc[c]
                
                n = randint(0, 1)
                
                if (n == 1):
                    arr1 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None)
                    arr2 = np.concatenate((enc1,np.concatenate((enc3,enc2),axis=None)),axis=None)
                    Y.append(np.array([1,0]))
                    Y.append(np.array([0,1]))
                else :
                    arr2 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None)
                    arr1 = np.concatenate((enc1,np.concatenate((enc3,enc2),axis=None)),axis=None)
                    Y.append(np.array([0,1]))
                    Y.append(np.array([1,0]))
                    
                X.append(arr1)
                X.append(arr2)

                
            j = j + 1

        yield (np.array(X),np.array(Y))

In [180]:
def image_generator_Evaluation(bs):
    
    j = 0
    while True:
        X = []
        Y = []
        
        while len(X) < bs:
            if (j >= numSamplesEval):
                j = 0
            else :
                a,b,c = validata[j]

                enc1 = features.iloc[a]
                enc2 = features.iloc[b]
                enc3 = features.iloc[c]
                
                n = randint(0, 1)
                
                if (n == 1):
                    arr1 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None)
                    arr2 = np.concatenate((enc1,np.concatenate((enc3,enc2),axis=None)),axis=None)
                    Y.append(np.array([1,0]))
                    Y.append(np.array([0,1]))
                else :
                    arr2 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None)
                    arr1 = np.concatenate((enc1,np.concatenate((enc3,enc2),axis=None)),axis=None)
                    Y.append(np.array([0,1]))
                    Y.append(np.array([1,0]))
                    
                X.append(arr1)
                X.append(arr2)

                
            j = j + 1

        yield (np.array(X),np.array(Y))

In [195]:
from tensorflow.keras import Sequential,layers, Input
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(6144,)))
model.add(Dropout(0.3))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))

In [196]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 256)               1573120   
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 2)                 1026      
Total params: 1,705,730
Trainable params: 1,705,730
Non-trainable params: 0
_________________________________________________________________


In [197]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [198]:
trainGen = image_generator(bs=BS)
testGen = image_generator_Evaluation(bs=BS)

In [ ]:
history = model.fit(
    trainGen,
    validation_data=testGen,
    steps_per_epoch=maxSamplesTrain // BS,
    validation_steps=maxSamplesEval // BS,
    epochs=30,
    )

## Abgabe

In [186]:
test = pd.read_fwf('test_triplets.txt', header=None, dtype=str)
numSamples = 59544
maxSamples = 59544

In [187]:
def image_generator_test(bs):
    
    j = 0
    while True:
        X = []
        Y = []
        
        while len(X) < bs:
            if (j >= numSamples):
                j = 0
            else :
                a,b,c = test.iloc[j]
                a = int(a)
                b = int(b)
                c = int(c)

                enc1 = features.iloc[a]
                enc2 = features.iloc[b]
                enc3 = features.iloc[c]
                
                arr1 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None) 
                X.append(arr1)

            j = j + 1

        yield (np.array(X))

In [188]:
testGen = image_generator_test(bs=BS)
pred = model.predict(testGen, steps=(maxSamples // BS),verbose=1)

1860/1860 [==============================] - 38s 21ms/step


In [189]:
final = []

for i in range(len(pred)):
    if (pred[i][0] >= 0.5):
        final.append(1)
    else:
        final.append(0)

In [190]:
el = 59520

while(el < maxSamples):
    a,b,c = test.iloc[el]
    a = int(a)
    b = int(b)
    c = int(c)

    enc1 = features.iloc[a]
    enc2 = features.iloc[b]
    enc3 = features.iloc[c]

    arr1 = np.concatenate((enc1,np.concatenate((enc2,enc3),axis=None)),axis=None)
    l = np.array(arr1).reshape(1,len(arr1))
    value = model.predict(l)
    
    if (value[0][0] >= 0.5):
        final.append(1)
    else:
        final.append(0)
    el = el + 1

In [191]:
finalfinal = np.array(final)
np.savetxt('predictions2.txt',finalfinal,fmt='%.0f')

### Prediction Statistics

In [192]:
preds = pd.read_fwf('predictions2.txt', header=None, dtype=str)
preds

,0
0,0
1,0
2,0
3,1
4,1
...,...
59539,1
59540,0
59541,1
59542,1


In [193]:
count = 0
for i,row in preds.iterrows():
    count = count + int(row[0])

In [194]:
count/numSamples*100

55.26165524654038